In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
processed_df = pd.read_pickle("data/processed_recipes.pkl")
processed_df

,name,ingredients,description,instructions,cuisine,course,diet,prep_time,categorized_prep_time,cleaned_ingredients,allergens,processed_name,processed_instructions,processed_description,allergen_type,processed_ingredients,combined_all_contexts,combined_name_ingredients
0,Doddapatre Tambuli Recipe (Karpooravalli Thayi...,"4 cups Indian borage (Doddapatre) , cleaned an...",Doddapatre Tambuli Recipe is a popular healing...,"To begin making the Doddapatre Tambuli, first ...",Indian,Side Dish,Vegetarian,30.0,~30 min,"[indian borage doddapatre, curd dahi yogurt, c...",[yogurt],doddapatre tambuli karpooravalli thayir pachadi,doddapatre tambuli doddapatre leave dry towel ...,doddapatre tambuli popular healing cold yoghur...,[Dairy],indian borage doddapatre curd dahi yogurt cumi...,doddapatre tambuli karpooravalli thayir pachad...,doddapatre tambuli karpooravalli thayir pachad...
1,Fish Tandoori Recipe,"1 Fish , nicely washed with bones on (I used P...",Have you ever tried out fish tandoori? Trust m...,"To begin making the Fish Tandoori recipe, keep...",North Indian,Appetizer,High Protein Non Vegetarian,80.0,~1 hour 30 min,"[fish, curd dahi yogurt, onion, green chilli, ...",[fish],fish tandoori,fish tandoori head fin inside fish many horizo...,fish tandoori delicious chicken tandoori tando...,[Fish and Shellfish],fish curd dahi yogurt onion green chilli tando...,fish tandoori fish curd dahi yogurt onion gree...,fish tandoori fish curd dahi yogurt onion gree...
2,Arbi Shimla Mirch Sabzi Recipe – Colocasia Cap...,300 grams Colocasia root (Arbi) 1 Green Bell ...,Arbi Shimla Mirch Sabzi Recipe is a variation ...,To begin making the Arbi Shimla Mirch Sabzi Re...,North Indian,Main Course,No Onion No Garlic (Sattvic),55.0,~1 hour,"[colocasia root arbi, green bell pepper capsic...",[peanut],arbi shimla mirch sabzi colocasia capsicum sabzi,arbi shimla mirch sabzi arbi pressure pressure...,arbi shimla mirch sabzi variation aloo capsicu...,"[Tree Nuts and Peanuts, Legumes and Seeds]",colocasia root arbi green bell pepper capsicum...,arbi shimla mirch sabzi colocasia capsicum sab...,arbi shimla mirch sabzi colocasia capsicum sab...
3,Ambur Style Brinjal Curry Recipe,5 Brinjal (Baingan / Eggplant) 1 Onion 2 Tom...,Brinjal Curry is prepared in different ways in...,To begin making the Ambur Style Brinjal Curry ...,Indian,Lunch,Vegetarian,20.0,~30 min,"[brinjal baingan eggplant, onion, ginger garli...","[sesame, peanut, mustard]",ambur style brinjal curry,ambur style brinjal curry oil heavy bottomed o...,brinjal curry different way different region s...,"[Legumes and Seeds, Tree Nuts and Peanuts]",brinjal baingan eggplant onion ginger garlic p...,ambur style brinjal curry brinjal baingan eggp...,ambur style brinjal curry brinjal baingan eggp...
4,Mavinakayi Menasinakai Curry Recipe - Raw Man...,"1 cup Mango (Raw) , diced 2 tablespoons Jagge...",Mangalorean Mavinakayi Menasinakai Curry Recip...,To begin making Mangalorean Mavinakayi Menasin...,Mangalorean,Side Dish,Vegetarian,35.0,~45 min,"[mango raw, jaggery, turmeric powder haldi, ta...",[sesame],mavinakayi menasinakai curry raw mango coconut...,mangalorean mavinakayi menasinakai curry raw m...,mangalorean mavinakayi menasinakai curry raw m...,[Legumes and Seeds],mango raw jaggery turmeric powder haldi tamari...,mavinakayi menasinakai curry raw mango coconut...,mavinakayi menasinakai curry raw mango coconut...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9491,Spinach Oats Vada Recipe,"Spinach Leaves (Palak), Chana dal (Bengal Gram...",Spinach Oats Vada Recipe is a healthy twist to...,"To prepare Spinach Oats Vada Recipe, in a larg...",South Indian,Snack,Vegetarian,50.0,~1 hour,"[spinach leave palak, chana dal bengal gram da...",[oats],spinach oats vada,spinach oats vada large mixing ingredient vada...,spinach oats vada healthy twist vada superfood...,[Grains and Gluten],spinach leave palak chana dal bengal gram dal ...,spinach oats vada spinach leave palak chana da...,sp

Train the recommendation model 

In [3]:
def get_recommendations(dish_name, df, best_num_topics, vectorizer, model, num_recommendations=5):
    vectorizer_matrix =  vectorizer.fit_transform(processed_df['processed_name'])

    # Train the model with the optimal number of topics
    called_model = model(n_components=best_num_topics, random_state=42)
    called_model.fit(vectorizer_matrix)
    topic_matrix = called_model.transform(vectorizer_matrix)# Topic distribution for each recipe

    # Add topic distributions to df
    for i in range(best_num_topics):
        processed_df[f'topic_{i}'] = topic_matrix[:, i]

    # Find the index of the given dish name
    recipe_index = df[df['name'].str.contains(dish_name, case=False, na=False)].index
    if not recipe_index.any():
        print("Dish not found.")
        return None
    
    recipe_index = recipe_index[0]  # Get the first match if there are multiple
    
    # Calculate cosine similarity for the topics
    cosine_similarities = cosine_similarity(topic_matrix)
    similar_indices = cosine_similarities[recipe_index].argsort()[-num_recommendations-1:-1][::-1]
    
    # Get the recommended recipes
    recommended_recipes = df.iloc[similar_indices].copy()
    recommended_recipes[ "similarity_score"] = cosine_similarities[recipe_index][similar_indices]

    return recommended_recipes[['name', 'similarity_score', 'cleaned_ingredients', 'cuisine', 'course', 'diet', 'allergen_type', 'categorized_prep_time']]

Get recipe recommendation using best NMF model on TF-IDF vectorizer

In [4]:
recommended_recipes_nmf_tfidf = get_recommendations("masala dosa", processed_df, 8, TfidfVectorizer(), NMF, 5)  
recommended_recipes_nmf_tfidf

,name,similarity_score,cleaned_ingredients,cuisine,course,diet,allergen_type,categorized_prep_time
5609,Mysore Masala Dosa Made With Ragi Dosa Batter,1.000000,"[idli rice, ragi seed, white urad dal whole, m...",Karnataka,South Indian Breakfast,Vegetarian,[Legumes and Seeds],~1 hour
5303,Italian Oat Crust Pizza Topped With Spinach An...,0.999425,"[whole wheat flour, instant oat oatmeal, lukew...",Italian,Lunch,Vegetarian,"[Grains and Gluten, Dairy]",~3.0 hours
9194,"Peas, Corn & Paneer Savory Hand Pies Recipe",0.999073,"[green peas matar, sweet corn, paneer homemade...",Fusion,Appetizer,Vegetarian,"[Dairy, Eggs]",~1 hour
281,Masala Akki Roti With Spinach Recipe - sarvapi...,0.999051,"[rice flour, water, spring onion green, spinac...",Andhra,South Indian Breakfast,Gluten Free,"[Dairy, Legumes and Seeds]",~1 hour 45 min
3804,Masala Akki Roti With Spinach Recipe - sarvapi...,0.999051,"[rice flour, water, spring onion green, spinac...",Andhra,South Indian Breakfast,Gluten Free,"[Dairy, Legumes and Seeds]",~1 hour 45 min


Get recipe recommendation using best SVD model on TF-IDF vectorizer

In [5]:
recommended_recipes_svd_tfidf = get_recommendations("masala dosa", processed_df, 15, TfidfVectorizer(), TruncatedSVD, 5)  
recommended_recipes_svd_tfidf

,name,similarity_score,cleaned_ingredients,cuisine,course,diet,allergen_type,categorized_prep_time
5609,Mysore Masala Dosa Made With Ragi Dosa Batter,1.000000,"[idli rice, ragi seed, white urad dal whole, m...",Karnataka,South Indian Breakfast,Vegetarian,[Legumes and Seeds],~1 hour
1189,Schezwan Masala Dosa Recipe Using Classic Idli...,0.997552,"[white urad dal whole, methi seed fenugreek se...",Fusion,Snack,Vegetarian,Unknown,~1 hour 15 min
6333,Schezwan Masala Dosa Recipe Using Classic Idli...,0.997552,"[white urad dal whole, methi seed fenugreek se...",Fusion,Snack,Vegetarian,[Legumes and Seeds],~1 hour 15 min
8393,Kollu Masala Dosa Recipe - High Protein Horse ...,0.995878,"[rice, horse gram dal kulith, methi seed fenug...",Konkan,South Indian Breakfast,Vegetarian,[Legumes and Seeds],~16.0 hours
2546,Kollu Masala Dosa Recipe - High Protein Horse ...,0.995878,"[rice, horse gram dal kulith, methi seed fenug...",Konkan,South Indian Breakfast,Vegetarian,[Legumes and Seeds],~16.0 hours


Get recipe recommendation using best NMF model on Count vectorizer

In [6]:
recommended_recipes_nmf_count = get_recommendations("masala dosa", processed_df, 24, CountVectorizer(), NMF, 5)  
recommended_recipes_nmf_count

,name,similarity_score,cleaned_ingredients,cuisine,course,diet,allergen_type,categorized_prep_time
5609,Mysore Masala Dosa Made With Ragi Dosa Batter,1.000000,"[idli rice, ragi seed, white urad dal whole, m...",Karnataka,South Indian Breakfast,Vegetarian,[Legumes and Seeds],~1 hour
6333,Schezwan Masala Dosa Recipe Using Classic Idli...,0.999689,"[white urad dal whole, methi seed fenugreek se...",Fusion,Snack,Vegetarian,[Legumes and Seeds],~1 hour 15 min
1189,Schezwan Masala Dosa Recipe Using Classic Idli...,0.999689,"[white urad dal whole, methi seed fenugreek se...",Fusion,Snack,Vegetarian,Unknown,~1 hour 15 min
8393,Kollu Masala Dosa Recipe - High Protein Horse ...,0.998316,"[rice, horse gram dal kulith, methi seed fenug...",Konkan,South Indian Breakfast,Vegetarian,[Legumes and Seeds],~16.0 hours
2546,Kollu Masala Dosa Recipe - High Protein Horse ...,0.998316,"[rice, horse gram dal kulith, methi seed fenug...",Konkan,South Indian Breakfast,Vegetarian,[Legumes and Seeds],~16.0 hours


Get recipe recommendation using best SVD model on Count vectorizer

In [7]:
recommended_recipes_svd_count = get_recommendations("masala dosa", processed_df, 3, CountVectorizer(), TruncatedSVD, 5)  
recommended_recipes_svd_count

,name,similarity_score,cleaned_ingredients,cuisine,course,diet,allergen_type,categorized_prep_time
746,Mysore Masala Dosa Made With Ragi Dosa Batter,1.000000,"[idli rice, special rice available indian stor...",Karnataka,South Indian Breakfast,Vegetarian,[Legumes and Seeds],~1 hour
6063,Whole Wheat Strawberry Banana Muffins Recipe,0.999989,"[whole wheat flour, caster sugar, nutmeg powde...",Continental,World Breakfast,Eggetarian,"[Grains and Gluten, Eggs, Dairy]",~45 min
3531,Foxtail and Jowar Millet Paniyaram Recipe,0.999971,"[foxtail millet, jowar seed, white urad dal wh...",Tamil Nadu,South Indian Breakfast,Diabetic Friendly,"[Grains and Gluten, Legumes and Seeds]",~1 hour
185,Foxtail and Jowar Millet Paniyaram Recipe,0.999971,"[foxtail millet, jowar seed, white urad dal wh...",Tamil Nadu,South Indian Breakfast,Diabetic Friendly,"[Grains and Gluten, Legumes and Seeds]",~1 hour
7669,Fresh Fig Compote Recipe For Desserts,0.999961,"[fresh fig, butter, nutmeg, brown sugar demera...",Continental,Dessert,Vegetarian,"[Grains and Gluten, Dairy]",~15 min


Filter the recommended recipes based on user preferences


In [8]:
# Define a function to filter recipes based on user preferences
def filter_recipes(df, cuisine=None, course=None, diet=None, prep_time=None, allergen_type=None):
    filtered_df = df
    if cuisine:
        filtered_df = filtered_df[filtered_df['cuisine'] == cuisine]
    if course:
        filtered_df = filtered_df[filtered_df['course'] == course]
    if diet:
        filtered_df = filtered_df[filtered_df['diet'] == diet]
    if prep_time:
        filtered_df = filtered_df[filtered_df['categorized_prep_time'] == prep_time]
    if allergen_type:
        # Exclude recipes with allergens in user preference
        filtered_df = filtered_df[~filtered_df['allergen_type'].apply(lambda x: bool(set(x) & set(allergen_type)))]
    return filtered_df

In [9]:
# Example 
recommended_recipes = recommended_recipes_nmf_tfidf
if recommended_recipes is not None:
    filtered_recommendations = filter_recipes(recommended_recipes, diet = "Vegetarian")
    display(filtered_recommendations)

,name,similarity_score,cleaned_ingredients,cuisine,course,diet,allergen_type,categorized_prep_time
5609,Mysore Masala Dosa Made With Ragi Dosa Batter,1.000000,"[idli rice, ragi seed, white urad dal whole, m...",Karnataka,South Indian Breakfast,Vegetarian,[Legumes and Seeds],~1 hour
5303,Italian Oat Crust Pizza Topped With Spinach An...,0.999425,"[whole wheat flour, instant oat oatmeal, lukew...",Italian,Lunch,Vegetarian,"[Grains and Gluten, Dairy]",~3.0 hours
9194,"Peas, Corn & Paneer Savory Hand Pies Recipe",0.999073,"[green peas matar, sweet corn, paneer homemade...",Fusion,Appetizer,Vegetarian,"[Dairy, Eggs]",~1 hour


In [21]:
import pickle
def get_recommendations_nmf_tfidf(dish_name, df, num_recommendations=5):

    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix =  tfidf_vectorizer.fit_transform(processed_df['processed_name'])

    # Train the model with the optimal number of topics
    best_num_topics = 8
    nmf_tfidf_model = NMF(n_components=best_num_topics, random_state=42)
    nmf_tfidf_model.fit(tfidf_matrix)
    topic_matrix = nmf_tfidf_model.transform(tfidf_matrix)# Topic distribution for each recipe

    # Save the models as pickle files in specified directory
    with open("models/nmf_tfidf_model.pkl", "wb") as nmf_tfidf_file:
        pickle.dump("nmf_tfidf_model", nmf_tfidf_file)
    with open("models/tfidf_vectorizer.pkl", "wb") as tfidf_file:
        pickle.dump("tfidf_vectorizer", tfidf_file)

    # Add topic distributions to df
    for i in range(best_num_topics):
        processed_df[f'topic_{i}'] = topic_matrix[:, i]

    # Find the index of the given dish name
    recipe_index = df[df['name'].str.contains(dish_name, case=False, na=False)].index
    if not recipe_index.any():
        print("Dish not found.")
        return None
    
    recipe_index = recipe_index[0]  # Get the first match if there are multiple
    
    # Calculate cosine similarity for the topics
    cosine_similarities = cosine_similarity(topic_matrix)
    similar_indices = cosine_similarities[recipe_index].argsort()[-num_recommendations-1:-1][::-1]
    
    # Get the recommended recipes
    recommended_recipes = df.iloc[similar_indices].copy()
    recommended_recipes[ "similarity_score"] = cosine_similarities[recipe_index][similar_indices]

    return recommended_recipes[['name', 'similarity_score', 'cleaned_ingredients', 'cuisine', 'course', 'diet', 'allergen_type', 'categorized_prep_time']]

In [22]:
recomended_recipe = get_recommendations_nmf_tfidf("masala dosa", processed_df, num_recommendations=5)
recomended_recipe

,name,similarity_score,cleaned_ingredients,cuisine,course,diet,allergen_type,categorized_prep_time
5609,Mysore Masala Dosa Made With Ragi Dosa Batter,1.000000,"[idli rice, ragi seed, white urad dal whole, m...",Karnataka,South Indian Breakfast,Vegetarian,[Legumes and Seeds],~1 hour
5303,Italian Oat Crust Pizza Topped With Spinach An...,0.999425,"[whole wheat flour, instant oat oatmeal, lukew...",Italian,Lunch,Vegetarian,"[Grains and Gluten, Dairy]",~3.0 hours
9194,"Peas, Corn & Paneer Savory Hand Pies Recipe",0.999073,"[green peas matar, sweet corn, paneer homemade...",Fusion,Appetizer,Vegetarian,"[Dairy, Eggs]",~1 hour
281,Masala Akki Roti With Spinach Recipe - sarvapi...,0.999051,"[rice flour, water, spring onion green, spinac...",Andhra,South Indian Breakfast,Gluten Free,"[Dairy, Legumes and Seeds]",~1 hour 45 min
3804,Masala Akki Roti With Spinach Recipe - sarvapi...,0.999051,"[rice flour, water, spring onion green, spinac...",Andhra,South Indian Breakfast,Gluten Free,"[Dairy, Legumes and Seeds]",~1 hour 45 min


In [12]:
#import pickle
#with open("models/nmf_tfidf_recom_model.pkl", "wb") as file:
#    pickle.dump("nmf_tfidf_recom_model.pkl", file)

In [ ]:
def get_recommendations_svd_tfidf(dish_name, df, num_recommendations=5):

    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix =  tfidf_vectorizer.fit_transform(processed_df['processed_name'])

    # Train the model with the optimal number of topics
    best_num_topics = 15
    svd_tfidf_model = TruncatedSVD(n_components=best_num_topics, random_state=42)
    svd_tfidf_model.fit(tfidf_matrix)
    topic_matrix = svd_tfidf_model.transform(tfidf_matrix)# Topic distribution for each recipe

    # Save the models as pickle files in specified directory
    with open("models/svd_tfidf_model.pkl", "wb") as nmf_tfidf_file:
        pickle.dump("svd_tfidf_model", nmf_tfidf_file)
    

    # Add topic distributions to df
    for i in range(best_num_topics):
        processed_df[f'topic_{i}'] = topic_matrix[:, i]

    # Find the index of the given dish name
    recipe_index = df[df['name'].str.contains(dish_name, case=False, na=False)].index
    if not recipe_index.any():
        print("Dish not found.")
        return None
    
    recipe_index = recipe_index[0]  # Get the first match if there are multiple
    
    # Calculate cosine similarity for the topics
    cosine_similarities = cosine_similarity(topic_matrix)
    similar_indices = cosine_similarities[recipe_index].argsort()[-num_recommendations-1:-1][::-1]
    
    # Get the recommended recipes
    recommended_recipes = df.iloc[similar_indices].copy()
    recommended_recipes[ "similarity_score"] = cosine_similarities[recipe_index][similar_indices]

    return recommended_recipes[['name', 'similarity_score', 'cleaned_ingredients', 'cuisine', 'course', 'diet', 'allergen_type', 'categorized_prep_time']]

In [24]:
recomended_recipe_svd_tfidf = get_recommendations_svd_tfidf("masala dosa", processed_df, num_recommendations=5)
recomended_recipe_svd_tfidf

,name,similarity_score,cleaned_ingredients,cuisine,course,diet,allergen_type,categorized_prep_time
5609,Mysore Masala Dosa Made With Ragi Dosa Batter,1.000000,"[idli rice, ragi seed, white urad dal whole, m...",Karnataka,South Indian Breakfast,Vegetarian,[Legumes and Seeds],~1 hour
1189,Schezwan Masala Dosa Recipe Using Classic Idli...,0.997552,"[white urad dal whole, methi seed fenugreek se...",Fusion,Snack,Vegetarian,Unknown,~1 hour 15 min
6333,Schezwan Masala Dosa Recipe Using Classic Idli...,0.997552,"[white urad dal whole, methi seed fenugreek se...",Fusion,Snack,Vegetarian,[Legumes and Seeds],~1 hour 15 min
8393,Kollu Masala Dosa Recipe - High Protein Horse ...,0.995878,"[rice, horse gram dal kulith, methi seed fenug...",Konkan,South Indian Breakfast,Vegetarian,[Legumes and Seeds],~16.0 hours
2546,Kollu Masala Dosa Recipe - High Protein Horse ...,0.995878,"[rice, horse gram dal kulith, methi seed fenug...",Konkan,South Indian Breakfast,Vegetarian,[Legumes and Seeds],~16.0 hours


In [ ]:
#import pickle
#with open("models/nmf_tfidf_recom_model.pkl", "wb") as file:
#    pickle.dump("nmf_tfidf_recom_model.pkl", file)

In [25]:
def get_recommendations_nmf_count(dish_name, df, num_recommendations=5):

    count_vectorizer = CountVectorizer()
    count_matrix =  count_vectorizer.fit_transform(processed_df['processed_name'])

    # Train the model with the optimal number of topics
    best_num_topics = 24
    nmf_count_model = NMF(n_components=best_num_topics, random_state=42)
    nmf_count_model.fit(count_matrix)
    topic_matrix = nmf_count_model.transform(count_matrix)# Topic distribution for each recipe

    # Save the models as pickle files in specified directory
    with open("models/nmf_count_model.pkl", "wb") as nmf_tfidf_file:
        pickle.dump("nmf_count_model", nmf_tfidf_file)
    with open("models/count_vectorizer.pkl", "wb") as count_file:
        pickle.dump("count_vectorizer", count_file)

    # Add topic distributions to df
    for i in range(best_num_topics):
        processed_df[f'topic_{i}'] = topic_matrix[:, i]

    # Find the index of the given dish name
    recipe_index = df[df['name'].str.contains(dish_name, case=False, na=False)].index
    if not recipe_index.any():
        print("Dish not found.")
        return None
    
    recipe_index = recipe_index[0]  # Get the first match if there are multiple
    
    # Calculate cosine similarity for the topics
    cosine_similarities = cosine_similarity(topic_matrix)
    similar_indices = cosine_similarities[recipe_index].argsort()[-num_recommendations-1:-1][::-1]
    
    # Get the recommended recipes
    recommended_recipes = df.iloc[similar_indices].copy()
    recommended_recipes[ "similarity_score"] = cosine_similarities[recipe_index][similar_indices]

    return recommended_recipes[['name', 'similarity_score', 'cleaned_ingredients', 'cuisine', 'course', 'diet', 'allergen_type', 'categorized_prep_time']]

In [26]:
recomended_recipe_nmf_count = get_recommendations_nmf_count("masala dosa", processed_df, num_recommendations=5)
recomended_recipe_nmf_count

,name,similarity_score,cleaned_ingredients,cuisine,course,diet,allergen_type,categorized_prep_time
5609,Mysore Masala Dosa Made With Ragi Dosa Batter,1.000000,"[idli rice, ragi seed, white urad dal whole, m...",Karnataka,South Indian Breakfast,Vegetarian,[Legumes and Seeds],~1 hour
6333,Schezwan Masala Dosa Recipe Using Classic Idli...,0.999689,"[white urad dal whole, methi seed fenugreek se...",Fusion,Snack,Vegetarian,[Legumes and Seeds],~1 hour 15 min
1189,Schezwan Masala Dosa Recipe Using Classic Idli...,0.999689,"[white urad dal whole, methi seed fenugreek se...",Fusion,Snack,Vegetarian,Unknown,~1 hour 15 min
8393,Kollu Masala Dosa Recipe - High Protein Horse ...,0.998316,"[rice, horse gram dal kulith, methi seed fenug...",Konkan,South Indian Breakfast,Vegetarian,[Legumes and Seeds],~16.0 hours
2546,Kollu Masala Dosa Recipe - High Protein Horse ...,0.998316,"[rice, horse gram dal kulith, methi seed fenug...",Konkan,South Indian Breakfast,Vegetarian,[Legumes and Seeds],~16.0 hours


In [ ]:
def get_recommendations_svd_count(dish_name, df, num_recommendations=5):

    count_vectorizer = CountVectorizer()
    count_matrix =  count_vectorizer.fit_transform(processed_df['processed_name'])

    # Train the model with the optimal number of topics
    best_num_topics = 24
    svd_count_model = TruncatedSVD(n_components=best_num_topics, random_state=42)
    svd_count_model.fit(count_matrix)
    topic_matrix = svd_count_model.transform(count_matrix)# Topic distribution for each recipe

    # Save the models as pickle files in specified directory
    with open("models/svd_count_model.pkl", "wb") as nmf_tfidf_file:
        pickle.dump("svd_count_model", nmf_tfidf_file)
    

    # Add topic distributions to df
    for i in range(best_num_topics):
        processed_df[f'topic_{i}'] = topic_matrix[:, i]

    # Find the index of the given dish name
    recipe_index = df[df['name'].str.contains(dish_name, case=False, na=False)].index
    if not recipe_index.any():
        print("Dish not found.")
        return None
    
    recipe_index = recipe_index[0]  # Get the first match if there are multiple
    
    # Calculate cosine similarity for the topics
    cosine_similarities = cosine_similarity(topic_matrix)
    similar_indices = cosine_similarities[recipe_index].argsort()[-num_recommendations-1:-1][::-1]
    
    # Get the recommended recipes
    recommended_recipes = df.iloc[similar_indices].copy()
    recommended_recipes[ "similarity_score"] = cosine_similarities[recipe_index][similar_indices]

    return recommended_recipes[['name', 'similarity_score', 'cleaned_ingredients', 'cuisine', 'course', 'diet', 'allergen_type', 'categorized_prep_time']]